# ch10. Combining data sources into a unified dataset
---
>- `raw` 데이터 처리하기
>- 데이터를 표현하는 파이썬 클래스 구현
>- 데이터를 PyTorch에서 사용할 수 있게 변환
>- train 데이터와 validation 데이터를 시각화

내려받은 CT 데이터 파일은 `.mhd` 파일과 `.raw`파일로 구성되있다. `.mhd` 파일은 메타 데이터 정보를 담고 있고 `.raw` 파일은 3D 배열로 만들어져 있다.

우리가 만들 `Ct` 클래스는 이 두가지 파일을 받아서 3차원 배열과 환자 좌표계를 index, row, column 좌표로 바꿔주는 matrix도 만든다.

annotations.csv 파일과 candidates.csv 파일을 들여다보자.

먼저 candidates.csv 파일에는 결절처럼 보이는 모든 조직 덩어리에 대한 정보가 들어있다.
551,000 줄이 `seriesuid`, `(X,Y,Z) coordinates`, `class`을 나타낸다. `class`가 1인 행이 결절 혹은 악성인 경우다. 1인 행은 1,351 개가 있다.

annotations.csv 파일에는 결절의 크기를 나타내는 `diameter_mm` 컬럼이 있다. 이를 이용해 결절 크기의 분포를 포함해서 train 데이터와 valid 데이터를 구성할 수 있다. 이런 값이 없는 상태로 valid 데이터를 만들면 극단적인 값만 가질 수 있다.

In [10]:
import pandas as pd

anno_path = 'F:\\공부/annotations.csv'
cand_path = 'F:\\공부/candidates.csv'

anno = pd.read_csv(anno_path)

anno

,seriesuid,coordX,coordY,coordZ,diameter_mm
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-128.699421,-175.319272,-298.387506,5.651471
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.783651,-211.925149,-227.121250,4.224708
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100398138793...,69.639017,-140.944586,876.374496,5.786348
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,-24.013824,192.102405,-391.081276,8.143262
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100621383016...,2.441547,172.464881,-405.493732,18.545150
...,...,...,...,...,...
1181,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-160.856298,-28.560349,-269.168728,5.053694
1182,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-102.189570,-73.865766,-220.536241,4.556101
1183,1.3.6.1.4.1.14519.5.2.1.6279.6001.994459772950...,-37.535409,64.041949,-127.687101,4.357368
1184,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,43.196112,74.438486,-200.523314,4.277203


In [9]:
cand = pd.read_csv(cand_path)

cand

,seriesuid,coordX,coordY,coordZ,class
0,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-56.08,-67.85,-311.92,0
1,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,53.21,-244.41,-245.17,0
2,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,103.66,-121.80,-286.62,0
3,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-33.66,-72.75,-308.41,0
4,1.3.6.1.4.1.14519.5.2.1.6279.6001.100225287222...,-32.25,-85.36,-362.51,0
...,...,...,...,...,...
551060,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-55.66,37.24,-110.42,0
551061,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,68.40,70.18,-109.72,0
551062,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,-82.29,-27.94,-106.92,0
551063,1.3.6.1.4.1.14519.5.2.1.6279.6001.997611074084...,38.26,83.50,-102.71,0


### train 셋과 valid 셋

어떤 지도 학습이든지 train 셋과 valid 셋으로 데이터를 나누게 된다. 다만 이 때 train 셋과 valid 셋 모두 실제 데이터를 대표하도록 만들어야 한다. 둘 중 하나라도 실제 데이터와 다르면 의도한 바와 다르게 모델이 작동할 수 있다.

결절을 구분하는 이 예제에서는 데이터를 크기 기준으로 정렬한다음, N번째 데이터를 valid 셋으로 분류할 계획이다. 이렇게 할 경우 주어진 데이터를 나타낼 수 있는 분포를 띌 것이다.

다만 문제가 있다. annotation.csv와 candidates.csv는 좌표값이 정확하게 들어맞지 않는다. 동일한 결절을 가리키는지 파악하기 위해서

`dsets.py`의 `getCandidateInfoList` 함수를 통해 데이터를 train 셋과 valid 셋으로 나눈다.


In [12]:
dsets.py

NameError: name 'dsets' is not defined